In [1]:
import pandas as pd
import os
from sqlalchemy import create_engine
from time import time

In [2]:

#Code taken from ingest_data.py, ran it locally. Replace variables below with correct parameters
user = 'username'
password = 'password'
host = 'localhost' 
port = 5432 
db = 'ny_taxi'
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{db}')


In [3]:
engine.connect()

In [4]:
#converted table_name and file_name to dictionary of lists, in order to iterate over multiple arguments

file_tablenames = [
    {'table_name': 'green_tripdata', 
     'file_name': 'green_tripdata_2019-09.csv.gz',
     #specify columns that are datetime in the CSV
     'date_column_names':['lpep_pickup_datetime','lpep_dropoff_datetime']
    },
    {'table_name': 'taxi_zone_lookup',
     'file_name': 'taxi+_zone_lookup.csv'
    }
]

#downloads file from URL given that it is a csv or csv.gz
  
for file_tablename in file_tablenames:
    file_name = file_tablename['file_name']
    table_name = file_tablename['table_name']


    df_iter = pd.read_csv(file_name, iterator=True, chunksize=100000)

    df = next(df_iter)

    #ensures columns are datetime if exist
    if 'date_column_names' in file_tablename:
        for column_name in file_tablename['date_column_names']: 
            df[column_name] = pd.to_datetime(df[column_name])
            
    df.head(n=0).to_sql(name=table_name, con=engine, if_exists='replace')

    df.to_sql(name=table_name, con=engine, if_exists='append')


    while True: 
        try:
            t_start = time()

            df = next(df_iter)            

            df.to_sql(name=table_name, con=engine, if_exists='append')

            t_end = time()

            print(f'{table_name}: inserted another chunk, took {(t_end - t_start):.2f} second' )

        except StopIteration:
            print(f"Finished ingesting data into {table_name}")
            break


green_tripdata: inserted another chunk, took 8.75 second
green_tripdata: inserted another chunk, took 8.49 second


c:\users\nour chaaban (acb)\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


green_tripdata: inserted another chunk, took 8.58 second
green_tripdata: inserted another chunk, took 3.56 second
Finished ingesting data into green_tripdata
Finished ingesting data into taxi_zone_lookup
